# Comparison of 3T and 7T data in epilepsy
-1. Concat zBrains outputs into dataframes
0. Demographics (Table 1)
1. zBrain/wBrain (surface)
    a. Histograms of vertex wise scores
        i. sub-comparisons with different smoothing kernels
    b. Quantifying extreme vertex groups
        i. number of identified abnormal areas
        ii. size of each abnormal area (number of adjacent extreme vertices)
2. Brainstats (surface)
    a. t-scores for 3T and 7T
    b. cohen's D map between 3T and 7T images

# -1. Prepare data

In [35]:
import sys
import importlib
import pandas as pd
import numpy as np
import vrtx

sys.path.append('/host/verges/tank/data/daniel/')  # Replace with the path to Utils
from Utils import gen
from Utils import id


In [50]:
importlib.reload(vrtx)
importlib.reload(id)

<module 'Utils.id' from '/host/verges/tank/data/daniel/Utils/id.py'>

In [77]:
# IDs of interest
dir_IDs = "/host/verges/tank/data/daniel/3T7T/z/data/pt/master_demo_19Mar2025_age_y.csv" # should be in long format
output_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs"
dir_tmp = "/host/verges/tank/data/daniel/3T7T/z/outputs/tmp"

MICs = {
    "name":"MICs",
    "dir_root": "/data/mica3/BIDS_MICs/derivatives",
    "ID_ctrl": ["HC"],
    "ID_pt": ["PX"]
}

PNI = {
    "name":"PNI",
    "dir_root": "/data/mica3/BIDS_PNI/derivatives",
    "ID_ctrl": ["PNC", "Pilot"],
    "ID_pt": ["PNE","PNA","PNF"]
}

studies = [MICs, PNI]


In [92]:
# zb output directories
# N.b. .surf.gii files usually in:
# root/zBrains_output/sub-ID/ses-xx/maps/region

dir_zb = "DM_zb_37comp" # that holds sub/ses
dir_sub = "maps"

cortex = {
    "region": "cortex",
    "surfaces": ["midthickness", "white"],
    "resolution": "32k",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    #"smoothing": [10]
    "smoothing": [2,5,10]
}

hippocampus = {
    "region": "hippocampus",
    "surfaces": ["midthickness"],
    "resolution": "0p5mm",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    #"smoothing": [5]
    "smoothing": [1,2,5]
}

subcortex = {
    "region": "subcortex",
    "features": ["ADC", "T1map", "thickness"],
    "smoothing": [2,5,10]
}

# regions = [cortex, hippocampus, subcortex]
regions = [cortex, hippocampus]

In [16]:
#determine file names to look for

file_ptrns = []  # Initialize as an empty list

for region in regions:
    region_ptrns = []  # Initialize as an empty list
    
    region_ptrns = vrtx.zbFilePtrn(region)

    # save patterns for each region in seperate list
    file_ptrns.append(region_ptrns)  # Append directly to the list

for region, i in zip(regions, range(len(file_ptrns))): print(f"{region['region']}: {len(file_ptrns[i])}")
print('\n'.join(' '.join(map(str,sl)) for sl in file_ptrns))
print() # Print the list of file patterns


cortex: 12
hippocampus: 6
subcortex: 3
hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-thickness_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-thickness_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-midthickness_feature-thickness_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-white_feature-ADC_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-white_feature-T1map_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-white_feature-thickness_smooth-10mm.func.gii
hemi-L_den-0p5mm_label-midthickness_feature-ADC_smooth-5mm.func.gii hemi-L_den-0p5mm_label-midt

In [80]:
# split IDs into 3T and 7T
IDptrn = sum([[study["ID_ctrl"] + study["ID_pt"]] for study in studies], [])
#print(IDptrn)
dir_splits = id.split(dir_IDs, "ID", IDptrn, dir_tmp)

[['HC', 'PX'], ['PNC', 'Pilot', 'PNE', 'PNA', 'PNF']]
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv (24 rows)PX: 16, HC: 8
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_2.csv (24 rows)PNE: 16, PNC: 8


In [93]:
# Agregate z-scores into df
for region, region_ptrns in zip(regions, file_ptrns):
    print("================================")
    region_name = region["region"]
    print(region_name)
    
    for ptrn in region_ptrns:
        print("--------------------------------")
        print(ptrn)
        
        for ID, study in zip(dir_splits, studies):
            study_name = study["name"]
            print(f"IDs from study: {study_name}")
            IDs = gen.fmt(ID, ["ID", "SES"])
            
            if region["resolution"] == "32k":
                len_vertex = 32492
            if region["resolution"] == "5k":
                len_vertex = 4842
            elif region["resolution"] == "0p5mm":
                len_vertex = 7262
            
            # initialize empty dataframe of size 32492 x len(IDs) with NaN numbers
            ID_SES = IDs["ID"] + "_" + IDs["SES"]
            df = pd.DataFrame(np.nan, index=range(len_vertex), columns=ID_SES)
            #print(df.shape)

            for i, s in zip(IDs["ID"], IDs["SES"]):
                i_s = i + "_" + s
                # get study root dir from ID name
                root_dir = id.get_rootDir(studyDirs,i)
                #print(f"\n\tID-SES: {i}-{s}")
                
                dir = "/".join([root_dir, dir_zb, i, s, dir_sub, region_name, i_s + "_" + ptrn]) # read in this path, save into a column in df
                # print(dir)

                if dir.endswith("csv"): 
                    # MUST BE REWORKED
                    vals =  vrtx.load_csv(dir)
                elif dir.endswith("func.gii"):
                    vals = vrtx.load_gifti(dir)
                else:
                    print(f"File extension unrecognized for file: {dir}")
                    continue

                if vals is None:
                    continue
                else:
                    df[i_s] = vals

            # save df
            out_name = study["name"] + "_" + ptrn.split(".")[0]
            out_path = f"{output_dir}/{study_name}/{region_name}/{out_name}.csv"
            df.to_csv(out_path)
            print(f"Saved {out_path}")


cortex
--------------------------------
hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii
IDs from study: MICs
[fmt] ID formatted
[fmt] SES formatted
[load_gifti] WARNING. Provided file does not exist: /data/mica3/BIDS_MICs/derivatives/DM_zb_37comp/sub-HC083/ses-02/maps/cortex/sub-HC083_ses-02_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii
[load_gifti] WARNING. Provided file does not exist: /data/mica3/BIDS_MICs/derivatives/DM_zb_37comp/sub-HC130/ses-02/maps/cortex/sub-HC130_ses-02_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii
Saved /host/verges/tank/data/daniel/3T7T/z/outputs/MICs/MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.csv
IDs from study: PNI
[fmt] ID formatted
[fmt] SES formatted
[load_gifti] WARNING. Provided file does not exist: /data/mica3/BIDS_PNI/derivatives/DM_zb_37comp/sub-PNC003/ses-03/maps/cortex/sub-PNC003_ses-03_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smoot

In [85]:
df

,sub-PNC003_ses-03,sub-PNC006_ses-03,sub-PNC011_ses-03,sub-PNC018_ses-01,sub-PNC019_ses-01,sub-PNC024_ses-a1,sub-PNC025_ses-a1,sub-PNC026_ses-a3,sub-PNE002_ses-a1,sub-PNE003_ses-a1,...,sub-PNE008_ses-a1,sub-PNE009_ses-a1,sub-PNE010_ses-a1,sub-PNE011_ses-a1,sub-PNE012_ses-a1,sub-PNE013_ses-a1,sub-PNE014_ses-a1,sub-PNE015_ses-a1,sub-PNE016_ses-a1,sub-PNE017_ses-a1
0,1.724776,NaN,1.639379,1.414373,1.419852,1.422327,1.868549,1.455297,1.617928,1.586744,...,1.321365,1.835166,1.496204,1.351454,1.412220,1.547825,1.451323,1.352028,1.666047,NaN
1,1.933225,NaN,1.905473,1.448825,1.560621,1.522741,2.050997,1.600324,1.793850,1.681802,...,1.447219,1.890193,1.576344,1.488830,1.572766,1.615586,1.731165,1.494990,1.721175,NaN
2,1.930469,NaN,1.921290,1.453811,1.571291,1.529945,2.046085,1.614684,1.805322,1.687969,...,1.457771,1.891576,1.594132,1.500232,1.583540,1.625574,1.750151,1.516219,1.730158,NaN
3,1.924514,NaN,1.933564,1.452417,1.585392,1.545254,2.038899,1.620549,1.814208,1.701382,...,1.469233,1.891585,1.608451,1.509353,1.592547,1.642725,1.773975,1.527174,1.745492,NaN
4,1.925181,NaN,1.931846,1.456764,1.580203,1.546643,2.041217,1.623331,1.814923,1.701804,...,1.469366,1.897696,1.610700,1.510612,1.593312,1.642662,1.765285,1.527683,1.739175,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7257,1.539636,NaN,1.504874,1.547334,1.557923,1.458298,1.767526,1.782843,1.602826,1.452004,...,1.438135,1.445546,1.542850,1.476716,1.437541,1.461388,1.458638,1.450855,1.678250,NaN
7258,1.517648,NaN,1.494753,1.541230,1.552955,1.447976,1.752863,1.765105,1.578050,1.448062,...,1.409707,1.426571,1.526032,1.476918,1.423590,1.451667,1.436353,1.445146,1.549881,NaN
7259,1.778628,NaN,1.724795,1.691298,1.717698,1.700630,1.974557,1.962939,1.817783,1.637644,...,1.597589,1.638443,1.746035,1.536344,1.721457,1.695065,1.714848,1.641786,2.075393,NaN
7260,1.747067,NaN,1.679349,1.675091,1.685605,1.665373,1.948088,1.925176,1.791552,1.600042,...,1.578945,1.613108,1.717604,1.525098,1.686238,1.650566,1.669866,1.611835,2.043177,NaN


In [ ]:
import re

# Example directory string
eg_dir = "/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC083/ses-02/maps/cortex/hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-2mm.func.gii"
print(eg_dir)

# Regular expressions to extract ID and session
match = re.search(r"sub-([^/]+)/ses-([^/]+)", eg_dir)

if match:
    ID = match.group(1)
    session = match.group(2)
    print(f"{ID}-{session}")
else:
    print("ID and session not found in the directory string.")

/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC083/ses-02/maps/cortex/hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-2mm.func.gii
HC083-02


## 1. zBrain
### a. Histograms
